# Практическая работа №4
### Выполнила студентка группы БВТ2101 Пьянова Анна Олеговна

Необходимо реализовать нейронную сеть вычисляющую результат логической операции согласно варианту. Затем реализовать функции, которые будут симулировать работу построенной модели. Функции должны принимать тензор входных данных и список весов. Должно быть реализовано 2 функции:
- Функция, в которой все операции реализованы как поэлементные операции над тензорами
- Функция, в которой все операции реализованы с использованием операций над тензорами из NumPy

Для проверки корректности работы функций необходимо:
- Инициализировать модель и получить из нее веса
- Прогнать датасет через не обученную модель и реализованные 2 функции.
- Сравнить результат.
- Обучить модель и получить веса после обучения
- Прогнать датасет через обученную модель и реализованные 2 функции. Сравнить результат.

Вариант 1: (a and b) or (a and c).

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from sklearn.metrics import accuracy_score
import numpy as np
import math

Создание датасета

In [2]:
inputs = np.array([
    [0, 0, 0],
    [0, 0, 1],
    [0, 1, 0],
    [0, 1, 1],
    [1, 0, 0],
    [1, 0, 1],
    [1, 1, 0],
    [1, 1, 1]
])
labels = ((inputs[:, 0] & inputs[:, 1]) | (inputs[:, 0] & inputs[:, 2])).reshape(-1, 1)

Создание модели нейронной сети и получение весов

In [22]:
model = models.Sequential()
model.add(layers.Dense(8, activation='relu', input_dim=3))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

initial_weights = model.get_weights()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Функция с поэлементными операциями

In [4]:
def simulate_model(inputs, weights):
    # Разделение весов на веса скрытого слоя и выходного слоя
    hidden_weights, hidden_bias, output_weights, output_bias = weights

    # Вычисление выхода скрытого слоя
    hidden_output = []
    for i in range(len(inputs)):
        hidden_layer_sums = []
        for j in range(len(hidden_weights)):
            hidden_layer_sum = sum(inputs[i][k] * hidden_weights[j][k] for k in range(len(inputs[i])))
            hidden_layer_sum += hidden_bias[j]
            hidden_layer_sums.append(max(0, hidden_layer_sum))
        hidden_output.append(hidden_layer_sums)

    # Вычисление выхода выходного слоя
    outputs = []
    for i in range(len(hidden_output)):
        output_sum = 0
        for j in range(len(hidden_output[i])):
            output_sum += hidden_output[i][j] * output_weights[j][0]
        output_sum += output_bias[0]
        output = 1 / (1 + math.exp(-output_sum))
        outputs.append(output)

    return [int(output > 0.5) for output in outputs]

Функция с использованием Numpy

In [5]:
def simulate_model_numpy(inputs, weights):
    # Разделение весов на веса скрытого слоя и выходного слоя
    hidden_weights, hidden_bias, output_weights, output_bias = weights

    # Вычисление выхода скрытого слоя
    hidden_output = np.dot(inputs, hidden_weights) + hidden_bias
    hidden_output = np.maximum(0, hidden_output)  # ReLU активация

    # Вычисление выхода выходного слоя
    output = np.dot(hidden_output, output_weights) + output_bias
    output = 1 / (1 + np.exp(-output))  # Sigmoid активация

    return output

Прогон датасета через необученную модель и две реализованные функции

In [18]:
initial_predictions = (model.predict(inputs) > 0.5).astype(int)
elementwise_predictions = simulate_model(inputs, initial_weights)
numpy_predictions = (simulate_model_numpy(inputs, initial_weights) > 0.5).astype(int)

print(f"Initial model accuracy: {accuracy_score(labels, initial_predictions)}")
print(f"Elementwise accuracy: {accuracy_score(labels, elementwise_predictions)}")
print(f"NumPy accuracy: {accuracy_score(labels, numpy_predictions)}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
Initial model accuracy: 0.5
Elementwise accuracy: 0.5
NumPy accuracy: 0.5


Обучение модели и получение весов

In [23]:
model.fit(inputs, labels, epochs=100, verbose=0)
trained_weights = model.get_weights()

Прогон датасета через обученную модель и две реализованные функции с полученными после обучения весами

In [24]:
trained_predictions = (model.predict(inputs) > 0.5).astype(int)
elementwise_predictions_trained = simulate_model(inputs, trained_weights)
numpy_predictions_trained = (simulate_model_numpy(inputs, trained_weights) > 0.5).astype(int)

print(f"Trained model prediction: {accuracy_score(labels, trained_predictions)}")
print(f"Elementwise prediction (trained): {accuracy_score(labels, elementwise_predictions_trained)}")
print(f"NumPy prediction (trained): {accuracy_score(labels, numpy_predictions_trained)}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Trained model prediction: 0.75
Elementwise prediction (trained): 0.375
NumPy prediction (trained): 0.75
